In [1]:
import tensorflow as tf
from elmo import elmo_embedding
import numpy as np
import os
from tqdm import tqdm
import json
from model_v2 import LanguageModel
import unicodedata
import re

In [2]:
with open('baomoi_punc/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('baomoi_punc/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
class2idx = {
    'fashion': 0,
    'technology': 1,
    'music': 2,
    'lifestyle': 3,
    'travel': 4
}
idx2class = {
    v: k for k, v in class2idx.items()
}

In [3]:
def cnn(inputs, labels, seq_lens, k_size, n_filters, n_classes, n_layers, n_units, dropout, l2_coef=1e-3, name='classifier', is_training=True, reuse=False):
    with tf.variable_scope(name, reuse=reuse):
        outputs = []
        mask = tf.sequence_mask(seq_lens, dtype=tf.float32)
        mask
        for ksz, nf in zip(k_size, n_filters):
            output = tf.layers.conv1d(inputs,
                                      nf, 
                                      ksz, 
                                      padding='same',
                                      kernel_initializer=tf.glorot_uniform_initializer(), 
                                      bias_initializer=tf.zeros_initializer(), 
                                      kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_coef))
            if is_training:
                output = tf.nn.dropout(output, 1-dropout, noise_shape=(1, 1, nf))
            output = tf.reduce_max(output, axis=1)
            outputs.append(output)
        outputs = tf.concat(outputs, axis=-1)
        for i in range(n_layers):
            W = tf.get_variable(name='W_{}'.format(i), shape=(outputs.shape[-1], n_units), trainable=True, initializer=tf.glorot_uniform_initializer(), regularizer=tf.contrib.layers.l2_regularizer(scale=l2_coef))
            b = tf.get_variable(name='b_{}'.format(i), shape=(n_units,), trainable=True, initializer=tf.zeros_initializer())
            outputs = tf.nn.xw_plus_b(outputs, W, b, name='dense_{}'.format(i))
            if is_training:
                outputs = tf.nn.dropout(outputs, keep_prob=1.0-dropout)
        W = tf.get_variable(name='W_out', shape=(outputs.shape[-1], n_classes), trainable=True, initializer=tf.glorot_uniform_initializer())
        b = tf.get_variable(name='b_out', shape=(n_classes,), trainable=True, initializer=tf.zeros_initializer())
        outputs = tf.nn.xw_plus_b(outputs, W, b, name='dense_out')
        pred = tf.argmax(outputs, axis=-1, name='predict', output_type=tf.int32)
        if is_training:
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=outputs))
            acc = tf.reduce_sum(tf.to_float(tf.equal(pred, labels))) / tf.to_float(tf.shape(inputs)[0])
            return outputs, pred, loss, acc
        else:
            prob = tf.nn.softmax(outputs, axis=-1)
            return outputs, pred, prob

In [4]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
x = tf.placeholder(dtype=tf.float32, shape=(None, None, 1024, 4), name='x')
y = tf.placeholder(dtype=tf.int32, shape=(None,), name='y')
seq_len = tf.placeholder(dtype=tf.int32, shape=(None,), name='seq_len')
drop = tf.placeholder(dtype=tf.float32, shape=(), name='drop')
elmo, elmo_l2_reg = elmo_embedding(x, seq_len, layer_norm=False, l2_coef=1e-3)
outputs, pred, prob = cnn(elmo, y, seq_len, (3, 4, 5), (128, 128, 128), 5, 2, 256, drop, 1e-3, name='classifier', is_training=False)
session.run(tf.global_variables_initializer())
class_saver = tf.train.Saver(tf.global_variables())

In [5]:
with open('15/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

lm_model = LanguageModel(**params, is_training=False, is_encoding=True)

lm_model.build_model()
lm_saver = tf.train.Saver([x for x in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'LanguageModel') if x not in tf.get_collection('LSTM_SAVED_STATE')])
session.run(tf.global_variables_initializer())
lm_saver.restore(session, '15/checkpoints/test/model.cpkt-315616')
class_saver.restore(session, 'classify data/12/classifier.cpkt-1960')

INFO:tensorflow:Restoring parameters from 15/checkpoints/test/model.cpkt-315616
INFO:tensorflow:Restoring parameters from classify data/12/classifier.cpkt-1960


In [6]:
def clean_text(x):
    x = str(unicodedata.normalize('NFKC', x.lower().strip()))
    return re.sub('\d+','N', re.sub('[ ]+',' ', re.sub('[\n\r][ \n\r]*',' L ', re.sub(r'(?P<punc>\W)',' \g<punc> ', x))))

def pad_sequence(words):
    maxlen = max(len(x) for x in words)
    arr = np.zeros(shape=(len(words), 1, maxlen))
    for ir in range(len(arr)):
        s = words[ir]
        arr[ir][0][:len(s)] = s
    return arr

def __embed_sequence(sentence):
    unk_char_idx = char2idx['U']
    sentence = [[char2idx.get(x, unk_char_idx) for x in word] for word in sentence]
    seq_len = len(sentence)
    inputs = pad_sequence(sentence)
    embeddings = session.run(lm_model.concated_timewise_output, feed_dict={
        lm_model.inputs: inputs, lm_model.seq_lens: [seq_len], lm_model.reset_state: True
    })
    return embeddings
def embed_sentence(sentence):
#     sentence = clean_text(sentence).split()
    return __embed_sequence(sentence)

In [7]:
def classify_text(sentence):
    sentence = clean_text(sentence).split()
    em = np.transpose(embed_sentence(sentence), (1, 0, 2, 3))
    pr = session.run(prob, feed_dict={
        x: em, seq_len: [len(em[0])], drop: 0.0
    })
    return {k: p for k, p in zip(class2idx, pr[0])}

In [8]:
classify_text("""Hôm nay Rb rất vui vì nhận đc giỏ quà xinh xắn nè: vòng Nguyệt Quế, chai dầu Ô Liu, mô hình tòa nhà truyền thống Hy Lạp, chiếc muỗng gỗ dễ thương, thông điệp đáng yêu và đặc biệt là món sữa chua phong cách Hy Lạp danh tiếng.

Cảm ơn Vinamilk rất nhiều về món quà dễ thương đậm chất "Hy Lạp" này nhé. Trải nghiệm đầu tiên của Rb sau khi nhâm nhi sữa chua Vinamilk Greek Yoghurt Style là độ sánh mịn đặc trưng kèm hương vị trái cây rất thơm ngon. Hơn nữa, sản phẩm này chứa gấp đôi protein và giảm 50% béo giúp cân bằng dinh dưỡng một cách tự nhiên, giúp cho da đẹp và dáng xinh mỗi ngày nèee.

Đợt nghỉ lễ sắp tới, cả nhà có thể bổ sung Vinamilk Greek Yoghurt Style vào thực đơn tráng miệng hoặc danh sách các món ăn xế nhé. Vừa ngon lại tốt cho sức khỏe mà không cần lo tăng cân nữa nhaaaa <3

#SữachuaHyLạp #VinamilkGreekYoghurt #Vinamilk #BíquyếtkhỏeđẹptừHyLạp #Bíquyếtkhỏeđẹphiệnđại #BữaXếThầnThánh""")

{'fashion': 0.00036384078,
 'technology': 7.625535e-07,
 'music': 1.792387e-07,
 'lifestyle': 0.5603025,
 'travel': 0.43933263}

In [9]:
classify_text("Bánh bèo nhập")

{'fashion': 0.00026679866,
 'technology': 0.094306804,
 'music': 0.09219778,
 'lifestyle': 0.57952005,
 'travel': 0.23370853}

In [10]:
classify_text("Phó Hằng, ka - ji - maaaaaaaaa :((((((")

{'fashion': 0.42557693,
 'technology': 0.0071761818,
 'music': 0.44346178,
 'lifestyle': 0.10953595,
 'travel': 0.014249128}

In [11]:
classify_text("Quán cf bất ổn trở lại rồi nèeeeee!!!")

{'fashion': 0.0034559632,
 'technology': 0.15520361,
 'music': 0.016786214,
 'lifestyle': 0.0010387475,
 'travel': 0.8235155}

In [12]:
classify_text("""Hôm qua ráng thức canh sale flash day 0h của Lazada mà loay hoay sao ngủ quên để lỡ mất.

Nay canh me 10h vào, trễ hơn người khác một xíu mà thấy deal đã vơi đi một nửa rồi.

Thôi lại đợi tới 20h để ráng hốt đợt cuối vậyyyyyy.

Từ giờ tới hết ngày vẫn còn sale đồng giá nhiều lắm, Rb thấy có cả máy tính bảng lenovo ideapad, đồng hồ wellington, tai nghe Beats,...

Bạn nào bị lỡ đợt sale trước thì tham khảo link bên dưới và canh giờ mua cho mình những sản phẩm mình thích với giá rẻ nhaaaaa ❤

👉🏻 http://bit.ly/8-8LZD""")

{'fashion': 5.3068256e-06,
 'technology': 0.81009513,
 'music': 0.0016731316,
 'lifestyle': 0.0012341929,
 'travel': 0.18699224}

In [13]:
classify_text("""Vòng vòngg""")

{'fashion': 0.0006011887,
 'technology': 0.01154514,
 'music': 0.0038009107,
 'lifestyle': 0.0004980784,
 'travel': 0.98355466}

In [14]:
classify_text('Mình bắt chước mèo kêu ko anh???')

{'fashion': 5.6284647e-05,
 'technology': 0.0631349,
 'music': 0.044590887,
 'lifestyle': 0.8834693,
 'travel': 0.008748686}

In [15]:
classify_text("""Tới giờ Ribi mới biết là trái sơ ri rẻ vậy thôi mà có chứa nhiều dưỡng chất lắm nha, có nhiều vitamin A, C, dưỡng chất đầy đủ, chống oxy hóa khiến da đẹp, không bị lão hoá và đi kèm rất nhiều lợi ích khác. Ăn sơ ri ngọt ngọt chua chua chấm muối nữa là đúng bài luôn :))

Bữa giờ đi quay, chụp liên tục, đang mệt tự nhiên được mua cho chai nước sơ ri siêu ngon, bạn ấy còn kêu nước này "thần thánh" lắm vì có chất Glutathione dưỡng sáng da từ bên trong nữa.

Mọi người muốn mua thì hỏi Kirin Ice+ Beauti, chai màu hồng dễ thương Ribi đang cầm trong hình đó. Uống sau khi ăn sáng hoặc ăn trưa để hấp thu tốt hơn, bổ sung nước và dưỡng chất cho da nha.

Đặc biệt vào ngày 02/08, Ribi được bên Kirin mời tham gia dự sự kiện ra mắt sản phẩm này ở Novotel Saigon, nên Ribi ưu ái dành tặng 20 suất cho 20 bạn may mắn đi cùng Ribi đến sự kiện và tận hưởng sản phẩm tuyệt vời này nha.

Đây là link tham gia nè: https://goo.gl/forms/7IILsh1kPUGWrKqz2. Form này sẽ đóng vào
cuối ngày 29 nhé.

Đúng ngày 30/7, những bạn may mắn sẽ được nhận email thông báo kết quả và hướng dẫn chi tiết để tham gia sự kiện nha.
Nhanh tay đăng kí nha mấy bạn""")

{'fashion': 1.7280232e-05,
 'technology': 0.00876589,
 'music': 2.3415252e-05,
 'lifestyle': 0.98334956,
 'travel': 0.0078438725}

In [16]:
classify_text("""Có ai mua đầm ở JM chưa nè??? Hãng thời trang chuyên về đầm thiết kế sẽ chính thức khai trương cơ sở thứ 15 tại quận 7 nè, mọi người tranh thủ ghé qua shopping nha, có nhiều đầm đẹp lắmmmm.

17h30 chiều mai (14/7/2018) tại JM - 320 Nguyễn Thị Thập, phường Tân Quy, quận 7 nhé mọi người.

Đến sớm sớm chút xíu để giao lưu cùng Rb và nhận thật nhiều ƯU ĐÃI từ JM các bạn nha ❤

#damthietkeJM""")

{'fashion': 0.9854525,
 'technology': 3.303245e-07,
 'music': 0.00010304647,
 'lifestyle': 0.00025467935,
 'travel': 0.014189452}

In [17]:
classify_text("""Nghìn năm hội ngộ :v""")

{'fashion': 0.0026618436,
 'technology': 0.7234123,
 'music': 0.15934226,
 'lifestyle': 0.05093748,
 'travel': 0.06364623}

In [18]:
classify_text("Vì phía trước hai ta sẽ còn đoạn đường rất dài !")

{'fashion': 0.017974585,
 'technology': 0.7119603,
 'music': 0.075174615,
 'lifestyle': 0.0068301233,
 'travel': 0.18806046}

In [19]:
classify_text("Quyen Le giỏi quá Q, bản lĩnh lắmmmmm, chúc mừng e và các đồng đội nha :*")

{'fashion': 0.0006057551,
 'technology': 0.87591195,
 'music': 0.101348825,
 'lifestyle': 0.000115391456,
 'travel': 0.02201808}

In [20]:
classify_text("❤ cảm ơn mọi người ! <3")

{'fashion': 0.55040735,
 'technology': 0.011474007,
 'music': 0.10431931,
 'lifestyle': 0.09591513,
 'travel': 0.23788427}

In [21]:
classify_text("Trailer ss5 nha các bạn trẻ :*")

{'fashion': 1.2542271e-05,
 'technology': 0.9948147,
 'music': 0.0035700693,
 'lifestyle': 2.2032264e-05,
 'travel': 0.0015805265}

In [22]:
classify_text("""Mấy bữa trước thấy bạn bè rần rần mua giày ở Sablanca nên nay được nghỉ cũng tranh thủ ra cửa hàng mua nè.
Hên sao đi mua trúng đợt offer 40% của cửa hàng nữa, mê quá nên tậu liền mấy đôi về luôn :))

Giày ở đây giá cả hợp lý, vừa túi tiền của mọi người nè, mẫu mã cũng đa dạng nữa, ngoài ra còn có nhiều mẫu túi xách xinh xinh cho các bạn thoải mái lựa chọn nhen.

Bạn nào đang phân vân chưa biết lựa giày ở đâu thì cứ ghé thử nha, biết đâu tìm đc đôi giày phù hợp cho mình nè 😁

Thông tin chi tiết về đợt OFFER 40% các bạn xem tại đây nhaa
👉🏻 https://www.facebook.com/Sablanca.vn/ nhaaaaaaaa""")

{'fashion': 0.017381525,
 'technology': 0.009179166,
 'music': 2.7035716e-05,
 'lifestyle': 0.5734099,
 'travel': 0.40000236}

In [23]:
classify_text("""Bạn nào là tín đồ làm đẹp chắc chắn biết sữa ong chúa rất tốt cho da đúng ko?
Được hôm đi siêu thị tìm được bộ sữa tắm và lotion này của Purité, có thành phần chứa tinh chất sữa ong chúa dưỡng trắng da toàn thân luôn, nhìn thấy mê quá nên Rb quyết định rinh ngay cả bộ về 😂

Đầu tiên là hơi bất ngờ với khả năng dưỡng trắng của em í, da k trắng liền mà rất tự nhiên, lên tone từ từ nè, da trắng rất tự nhiên chứ không phải kiểu trắng bệch nha. Rb có đọc được là nhờ thành phần sữa ong chúa tự nhiên rất giàu vitamin giúp kích thích quá trình thay da nên rất an toàn nhen. Đây cũng là cơ chế dưỡng trắng tự nhiên mà Rb vừa tìm hiểu được.

Thứ hai là ai đã dùng sản phẩm của dòng này rồi đều phải công nhận hương thơm rất dễ chịu, nhẹ nhàng, tinh tế đúng k? Nếu dùng kèm với em lotion cùng bộ nữa thì hiệu quả dưỡng trắng tăng gấp đôi và không cần dùng thêm nước hoa nữa luôn đó.

Lotion này không bị nhờn rít, thoa tới đâu thấm tới đó, rất thích hợp cho mùa hè năng động nèeee.

Vừa trắng tự nhiên an toàn, vừa thơm tinh tế, chắc chắn sẽ là sự lựa chọn phù hợp cho các bạn gái tụi mình đó ☺️ 😉

Các bạn có thể tìm hiểu kĩ sản phẩm hơn ở link dưới nhaaa
👉🏻 bit.ly/RS-PuriteWhitening""")

{'fashion': 5.1378515e-07,
 'technology': 3.79505e-07,
 'music': 2.8193603e-10,
 'lifestyle': 0.99999905,
 'travel': 1.4747675e-07}

In [24]:
classify_text("Vẫn đang quay khuyaaa nèe")

{'fashion': 4.3344113e-05,
 'technology': 0.57065797,
 'music': 0.418213,
 'lifestyle': 3.7746738e-06,
 'travel': 0.011081896}

In [25]:
classify_text("Nhiễm phim phải nhập vai vào phim luôn mới trất'ss nhé =)))))))")

{'fashion': 0.0018696897,
 'technology': 0.6213504,
 'music': 0.34524873,
 'lifestyle': 0.02408247,
 'travel': 0.0074486285}

In [26]:
classify_text("""Như trong bài chia sẻ trước của Rb, nước gạo Morning Rice rất tốt cho sức khoẻ chúng ta rồi đúng không?
Morning Rice là sản phẩm được làm từ gạo, và nguyên liệu gạo xuất xứ 100% từ Hàn Quốc, sản phẩm cũng bán chạy nhất tại Hàn , lại có vị thơm, ngọt dịu mà không bị ngấy.

Rb thường uống Morning Rice khi đi quay, hay bất cứ khi nào thèm 1 một món nước nào ngọt ngọt, đặc biệt nếu các bạn uống Morning Rice vào buổi sáng cực kỳ tốt luôn, không những cung cấp đủ nước cho cả một ngày dài mà còn tốt cho sức khoẻ, giúp sảng khoái tinh thần cho 1 ngày mới.

Bạn nào đã sử dụng sản phẩm này thường xuyên và thấy hiệu quả rồi thì cùng chia sẻ cùng Rb cho các bạn chưa dùng nha 😀""")

{'fashion': 5.744237e-08,
 'technology': 1.9635233e-06,
 'music': 8.4623827e-07,
 'lifestyle': 0.9999598,
 'travel': 3.7258942e-05}

In [27]:
classify_text("""Lâu lắm mới đc chụp chân dung :))""")

{'fashion': 0.0028528257,
 'technology': 0.86728126,
 'music': 0.04706661,
 'lifestyle': 0.021103393,
 'travel': 0.061695933}

In [28]:
classify_text("""Đlq nhưng phiên bản mở rộng của Global Warming của con Pit sẽ được ra mắt vào 25/11 năm nay bổ sung 1 số bài mới trong đó có Timber feat Flop-sa. 

Cover chính thức nhé, cứ tựa tựa như mụ Nicki với cái Era Pink Friday 8-}""")

{'fashion': 6.6393713e-09,
 'technology': 0.99877745,
 'music': 0.0012156122,
 'lifestyle': 1.5217615e-06,
 'travel': 5.4059547e-06}

In [29]:
classify_text("""Sinh nhật năm nay thực sự quá hạnh phúc, nhận được rất nhiều lời chúc của gia đình, Faptv, bạn bè, rất nhiều tình cảm từ những người xa lạ,..
Cảm ơn tất cả các bạn đã chúc sn và gây rất nhiều bất ngờ cho Rb nha, xúc động lắm lắmmm.
Có nhiều tin nhắn quá Rb k cảm ơn hết từng người được nên cảm ơn các bạn tại stt này nha.
Hy vọng những năm về sau nữa chúng ta vẫn mãi bên nhau thế này ❤

P/s: Đây là nhóm bạn kim cương 10 năm tổ chức sn cho mình """)

{'fashion': 0.07306688,
 'technology': 0.0027055887,
 'music': 0.57545847,
 'lifestyle': 0.0005227887,
 'travel': 0.34824622}

In [30]:
classify_text("Chỉ muốn nói là hôm nay quá nhiều bất ngờ và hạnh phúc. Một ngày tuyệt đẹp <3")

{'fashion': 0.3468721,
 'technology': 0.014884135,
 'music': 0.48088226,
 'lifestyle': 0.14248663,
 'travel': 0.014874797}

In [31]:
classify_text("Lâu lắm mới cột tóc cao :))")

{'fashion': 0.03522759,
 'technology': 0.20556141,
 'music': 0.047811918,
 'lifestyle': 0.38692945,
 'travel': 0.32446963}

In [32]:
classify_text("""Lâu lắm mới làm clip nè :))

#Sponsored #SpotifyMVP #Spotify #SpotifyNgheLaMe #SpotifyVietnam""")

{'fashion': 0.0021324586,
 'technology': 0.028690005,
 'music': 0.00039155222,
 'lifestyle': 0.010334127,
 'travel': 0.9584518}

In [33]:
classify_text("""Lâu lắm mới làm clip nè :))""")

{'fashion': 0.0021030519,
 'technology': 0.14405298,
 'music': 0.17434731,
 'lifestyle': 0.4145741,
 'travel': 0.26492256}

In [34]:
classify_text("""Vừa được bỏ túi bí kíp chống nắng trong những ngày hè oi bức, vừa có quà, ai thích không nè!

Có rất nhiều loại kem chống nắng, nhưng để lựa được loại kem chống nắng phù hợp, không gây nhờn rít và sử dụng phù hợp lại không dễ chút nào. Rb mới biết thêm kem chống nắng Anthelios Dry Touch, nếu bạn nào dùng rồi chắc chắn rất thích đúng k?

Rb dùng trước khi ra ngoài hay trước khi make up, apply một lớp kem chống nắng này, sau nhiều giờ hoạt động ngoài trời da không hề bị đổ dầu và bã nhờn nhờ có hoạt chất Airlicium. Bên cạnh đó, nhờ có công nghệ độc quyền XL- Protect giúp chống nắng rất hiệu quả, ngăn chặn được tia UVB, UVA đặc biệt là tia hồng ngoại và ô nhiễm.

Ribi còn phát hiện ra La Roche-Posay có hẳn dòng xịt chống nắng Anthelios Mist, khi xịt lên da thì thẩm thấu nhanh và không hề đọng giọt. Và đặc biệt là có thể xịt ngay trên lớp makeup luôn nhé, rất tiện cho công việc của Rb khi phải quay liên tục ngoài trời và k có nhiều thời gian chăm sóc lại lớp chống nắng và make up của mình.

Cuối cùng là sau một ngày làm việc thì đừng ai quên phải tẩy trang trước khi đi ngủ nha. Đừng lười mà đi ngủ luôn, mụn sẽ nổi đầy mặt đó. Các bạn hãy dùng 1 miếng bông tẩy trang và cho 1 ít nước tẩy trang Micellar Water Ultra lên và làm sạch da. Với công nghệ cải tiến Glyco Micellar giúp lấy đi bụi bẩn, bã nhờn và lớp trang điểm, giúp làm sạch da sâu vượt trội, nhưng không hề có cảm giác khô căng, da sạch rồi nhennn.

Mọi người muốn biết thêm thông tin thì xem tại link đây nha: http://bit.ly/SUN_LZDKOL Đang có ưu đãi khi mua sản phẩm La Roche Posay trên Lazada đó!

Bên cạnh đó thì bạn nào muốn có quà thì nhớ nhanh tay like fanpage La Roche-Posay Vietnam và post này của Ribi nha. Sau đó tag tên 3 người bạn thân của mình, Ribi sẽ lựa chọn ngẫu nhiên 5 bạn may mắn nhận được bộ sản phẩm như này nhé ❤

#larocheposay #anthelios #baovetoiuu #dakhoedonnang""")

{'fashion': 1.9522306e-06,
 'technology': 0.00015338778,
 'music': 3.443241e-07,
 'lifestyle': 0.99965286,
 'travel': 0.00019136645}

In [35]:
classify_text("""Sáng tới giờ chốt đơn cho mấy xị đẹp đi chơi lễ mà con bé cũng háo hức theo😝😝 lễ này mấy xị đi chơi ảnh đẹp thì tặng bé nha, bé nằm nhà chờ tải ảnh đẹp của mấy xị💓💓💓""")

{'fashion': 0.002549346,
 'technology': 0.0025683732,
 'music': 0.00090067496,
 'lifestyle': 0.03395762,
 'travel': 0.960024}

In [36]:
classify_text("""Hơn 100 đơn hàng mỗi ngày là nhờ vào chất lượng và tình cảm khách hàng tiếng lành đồn xa😜 cảm ơn khách hàng luôn ủng hộ mẹ Joy!!! Sẽ cố gắng trong thời gian sớm nhất thêm 1 chi nhánh nữa để chị em gần hơn với Tóc Giả HMT 🙏🏻""")

{'fashion': 0.0715459,
 'technology': 0.41376218,
 'music': 0.050756563,
 'lifestyle': 0.3171401,
 'travel': 0.14679521}

In [37]:
classify_text("""👉🏻A08 vẫn cứ là tuyệt vời nên pr lại cho mấy chị em nhà mình đừng bỏ Quên em ấy😜😜 1 bước xinh lung linh không phải rườm rà gì hết ý💚💚💚
5màu: đen, nâu đen, nâu đỏ, nâu vàng, socola""")

{'fashion': 0.062238663,
 'technology': 0.0080154985,
 'music': 0.0016431166,
 'lifestyle': 0.9206771,
 'travel': 0.0074255844}

In [38]:
classify_text("""Là cô gái hay mộng mơ😹😹""")

{'fashion': 0.0029898842,
 'technology': 3.7148387e-05,
 'music': 0.0011402399,
 'lifestyle': 0.99566627,
 'travel': 0.00016646052}

In [39]:
classify_text("""Không có việc gì khó, chỉ sợ lòng không bền""")

{'fashion': 0.017157283,
 'technology': 0.8745693,
 'music': 0.07380007,
 'lifestyle': 0.025712987,
 'travel': 0.008760272}

In [40]:
classify_text("""Nếu bạn thích chinh phục những cung đường mạo hiểm thì đây chính là gói bảo hiểm dành cho bạn. Bảo hiểm FWD đang tặng MIỄN PHÍ 100,000 gói bảo hiểm tai nạn cá nhân, bảo hiểm đến. Nhanh tay đăng ký và nhận ngay gói bảo hiểm chỉ trong vòng 3 phút tại""")

{'fashion': 2.5013337e-06,
 'technology': 0.9803263,
 'music': 8.165938e-06,
 'lifestyle': 0.00016041716,
 'travel': 0.019502576}